# General Filter

Collated Sim Data

In [2]:
import pandas as pd
import numpy as np

sim_data=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\INPUT_FILES\sim_data.csv',usecols=('ShortId','Data'),low_memory=False)

# sim_data=sim_data[sim_data['Title'].str.contains('Comment')==True]
df=sim_data.groupby(['ShortId'])['Data'].apply(lambda x: ','.join(x)).reset_index()


df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\Collated_sim_data.csv',index=False)
df

,ShortId,Data
0,301426600,Contact ID:ARQQ1IKKBADT5\r\n\r\nDA Details : ...
1,301426710,Contact ID: A1L7MZTUOQ2LX3\r\n\r\nDA Details :...
2,301428954,Emailed DSP to provide infraction information....
3,301432697,Contact ID:A11CAHDPX03HDK\r\n\r\nDuplicate tic...
4,301433225,Packages exposed to weather and not ringing do...
...,...,...
3499,E105951900,This package was delivered to a wrong address ...
3500,E209355059,// Ticket details over 2500 characters //\n(@l...
3501,E209525137,- Customer indicated in A3SX3NQA761WXK that th...
3502,E209528101,ECR - Please send initial Tier 1 infraction no...


Collated Infractions data final

In [3]:
df1=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\Collated_sim_data.csv',usecols=('ShortId','Data'))
df1['ShortId']=df1['ShortId'].astype(str)

audit_query=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\INPUT_FILES\audit_query.csv',usecols=('case_id','details','root_cause','tid'),encoding='ISO-8859-1')

audit_query_1 = audit_query.rename(columns = {"case_id": "ShortId"}) 

audit_query_1['ShortId'].dropna(inplace=True)


x=audit_query_1.merge(df1,on='ShortId',how='left')
x['Data'] = x.Data.fillna('No data available')

x = x[(x['root_cause'] != 'Positive Feedback')&(x['root_cause'] != 'Offboarding - Combined Threshold')]

x['Infraction']=x['Data'].apply(lambda st: st[st.find("Infraction")+11:st.find("Additional Details")])
x['Additional_Details']=x['Data'].apply(lambda st: st[st.find("Additional Details:"):st.find("Delivery Tip")])

x=x[x['Data'].str.contains('Appeal Successful|Appeal Rejected|Overturned during MTI process|DSP DA station tickets are now processed through RIVER')==False]

x.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\Collated_Infractions_data_final.csv',index=False)
x

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details
0,312952902,Your login (without @amazon): athbechi Co...,PPE Feedback,TBA023050801801,Reaching out to customer for evidence in conta...,ut to customer for evidence in contact A3O4IDG...,
1,312963401,Copy and paste the Amazon Flex delivery partne...,T2 - Behavioral - Unprofessional behavior towa...,NaN,Driver is a DA. Flipping to correct CTI. \r\n\...,"DA reputation - Behavioral - Insulting, unpro...",
2,313036436,"Delivery Station (Station Code, City, State): ...",Not a Tier Infraction,TBA023347308401,No data available,ailabl,
3,312949075,Your login (without @amazon): kimmears Co...,T2 - Unsafe driving behavior - Property Damage,TBA022025095601,No data available,ailabl,
4,313921298,"Delivery Station (Station Code, City, State): ...",T2 - Package Mishandling - Not following Deliv...,TBA031053734101,Contact ID:A2ITMAR6LEO9R\r\n\r\nDuplicate tick...,Tier 2 DA reputation - Package Mishandling - N...,Additional Details:Customer complains that dri...
...,...,...,...,...,...,...,...
3618,312226721,"Delivery Station (Station Code, City, State): ...",Not a Tier Infraction,NaN,No data available,ailabl,
3619,E105336722,NaN,T2 - Behavioral - Unprofessional behavior towa...,NaN,Security manager called to report the behavior...,"DA reputation - Behavioral - Insulting, unpro...",Additional Details: The customer got aggressiv...
3620,E104942601,NaN,T1 - Behavioral - Theft of Property,NaN,* This ticket will automatically be set to Ass...,"\r\nHello Cornucopia Logistics LLC,\r\n\r\nYou...",Additional Details: We received 2 complaints f...
3621,E209528101,NaN,T1 - Behavioral - Theft of Property,NaN,ECR - Please send initial Tier 1 infraction no...,"\r\n\r\n\r\nHello Inter-Citi Logistics,\r\n\r...",Additional Details: Theft of package\r\n\r\nIf...


# Match Infractions - First check

In [4]:
df3=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\Collated_Infractions_data_final.csv')


df3['root_cause']=df3['root_cause'].str.lower() 
df3['Infraction']=df3['Infraction'].str.lower()
df3['Data']=df3['Data'].str.lower() 



word_list=['badge','bags left','behavior towards a customer','business closed','dangerous','delivered to receiver','not following','did not follow sop','headphones','incorrect scan',
    'inside lobby','lawn','leaving keys','left in driveway','left in road','left unattended','locker','mail box','mailroom/leasing','no safe location',
    'non-swa','on/near garbage','ovesize','safety vest','seat belt','station employee','station standard work compliance','tailgating','throwing','unable to access',
    'unauthorized entry','under the roof','unsafe driving at station','property damage','unsecure packages']

df3['Infraction1'] = df3.root_cause.str.extract('({})'.format('|'.join(word_list)), 
                        expand=False).str.lower()

df3['Infraction2'] = df3.Infraction.str.extract('({})'.format('|'.join(word_list)), 
                        expand=False).str.lower()

df3['First_Status']=np.where(df3['Infraction1']==df3['Infraction2'],'True','NS')

df4_1=df3[df3['Data'].str.contains('Appeal Successful|Appeal Rejected|Overturned during MTI process|DSP DA station tickets are now processed through RIVER')==False]
df4_1['Infraction'] = df4_1['Infraction'].str.strip()

# z=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\INPUT_FILES\comp_data.csv',encoding='ISO-8859-1')
# y=df4_1.merge(z,left_on='tid',right_on='Tracking Id',how='left')

df4_1.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv',index = False)

df4_1

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status
0,312952902,Your login (without @amazon): athbechi Co...,ppe feedback,TBA023050801801,reaching out to customer for evidence in conta...,ut to customer for evidence in contact a3o4idg...,NaN,NaN,NaN,NS
1,312963401,Copy and paste the Amazon Flex delivery partne...,t2 - behavioral - unprofessional behavior towa...,NaN,driver is a da. flipping to correct cti. \r\n\...,"da reputation - behavioral - insulting, unprof...",NaN,behavior towards a customer,NaN,NS
2,313036436,"Delivery Station (Station Code, City, State): ...",not a tier infraction,TBA023347308401,no data available,ailabl,NaN,NaN,NaN,NS
3,312949075,Your login (without @amazon): kimmears Co...,t2 - unsafe driving behavior - property damage,TBA022025095601,no data available,ailabl,NaN,property damage,NaN,NS
4,313921298,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - not following deliv...,TBA031053734101,contact id:a2itmar6leo9r\r\n\r\nduplicate tick...,tier 2 da reputation - package mishandling - n...,Additional Details:Customer complains that dri...,not following,not following,True
...,...,...,...,...,...,...,...,...,...,...
3552,312226721,"Delivery Station (Station Code, City, State): ...",not a tier infraction,NaN,no data available,ailabl,NaN,NaN,NaN,NS
3553,E105336722,NaN,t2 - behavioral - unprofessional behavior towa...,NaN,security manager called to report the behavior...,"da reputation - behavioral - insulting, unprof...",Additional Details: The customer got aggressiv...,behavior towards a customer,NaN,NS
3554,E104942601,NaN,t1 - behavioral - theft of property,NaN,* this ticket will automatically be set to ass...,"\r\nhello cornucopia logistics llc,\r\n\r\nyou...",Additional Details: We received 2 complaints f...,NaN,NaN,NS
3555,E209528101,NaN,t1 - behavioral - theft of property,NaN,ecr - please send initial tier 1 infraction no...,"\r\n\r\n\r\nhello inter-citi logistics,\r\n\r\...",Additional Details: Theft of package\r\n\r\nIf...,NaN,NaN,NS


# Dumping packages inside lobby front entrance

In [5]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('dumping packages inside lobby/front entrance')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()


df['Second_Status']=(df['Additional_Details'].str.contains('drop|leaving|stash|left|deliver|dumped|leave|abandon|sent'))&(df['Additional_Details'].str.contains('package|item|box| '))&(df['Additional_Details'].str.contains('unsafe|outside|at|between|in|by|behind|of'))&(df['Additional_Details'].str.contains('fence|stairwell on the first floor|apartment front door|apartment door|mailbox|mailboxes|leasing|front in the steps|lobby|entrance|building|bldg|front door of the apartment|front of elevator|front door of apartment|the door|store|gate|public hall|building|main door|steps|mailroom'))


df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Dumping packages inside lobby.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
30,313901161,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA029511730901,"as per contact id:a27ura2syl3n5l ,this morning...",tier 2 da reputation - behavioral \u2013 dumpi...,additional details:property manager complainin...,inside lobby,inside lobby,True,True
79,313939064,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA029609715901,as per contact id: the driver left the package...,da reputation - behavioral \u2013 dumping pack...,additional details: the driver left the packa...,inside lobby,inside lobby,True,True
92,313905778,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA030707218801,contact id: a3w40mxpcdfhb6\r\n \r\nduplicate t...,da reputation - behavioral \u2013 dumping pack...,additional details: customer : \the boxes have...,inside lobby,inside lobby,True,True
103,313937150,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA029936312001,contact id:a1y1y6d03prs9\r\n\r\nduplicate tick...,da reputation - behavioral \u2013 dumping pack...,additional details: driver dumped packages in ...,inside lobby,inside lobby,True,True
149,313902638,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA222113060000,contact id: a2fp1ph97wlk11\r\n\r\nduplicate ti...,da reputation - behavioral \u2013 dumping pack...,additional details: customer is complaining th...,inside lobby,inside lobby,True,True
...,...,...,...,...,...,...,...,...,...,...,...
3272,313623888,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA028149572801,contact id: a1ugzqn6l3roxg\r\n\r\nattached pod...,da reputation - behavioral \u2013 dumping pack...,additional details: driver left the package in...,inside lobby,inside lobby,True,True
3375,314359907,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA033182348201,no data available,ailabl,NaN,inside lobby,NaN,NS,False
3438,314353700,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA029093392601,no data available,ailabl,NaN,inside lobby,NaN,NS,False
3523,314166913,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages inside lobb...,TBA222619733000,contact id: a2doj46nvndkvc\r\n\r\nduplicate ti...,da reputation - behavioral \u2013 dumping pack...,additional details: customer states that the d...,inside lobby,inside lobby,True,True


# Dumping packages outside locker

In [6]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('dumping packages outside locker')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()


df['Second_Status']=df['Additional_Details'].str.contains('left the packages in the mail room instead of delivering the package in the locker|dumped all these packages in the luxer room|left by the door inside the luxer room|left the package lean up against the lockers|into the locker room without being secured in a locker|outside the package room instead of|dumped the packages without scanning them in the luxer lockers|left in front of the locker|left the packages out of the locker system|left the packages unsecured on the parcel room|left the package in the mail-room|packages are supposed to go into the hub lockers|dumped these packages at the amazon hub|dumped all packages on his route for this address in the locker room on the ground unattended|left the package in the mail room without putting them in the locker|did not use the luxor locker|locker room floor|dump packages in the lobby|left outside of  locker|outside at the locker|instead of locker|outside of the hub|outside the hub|outside of locker|outside the locker|left on the mailroom floor|next to the locker|out side the locker|outside a locker|outside of the locker|left outside of the parcel pending locker|outside of the Luxer room|Not delivering to the locker|outside the hub locker|on top of the locker|outside of the HUB|outside locker|leave packages out of the HUB|on floor of locker|outside the Amazon Hub|outside the Hub|outside of the Delivery Locker|on the floor in parcel locker room|locker room unattended|unattended in the mail room')



df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Dumping packages outside locker.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
129,313929916,Copy and paste the Amazon Flex delivery partne...,t2 - behavioral - dumping packages outside locker,TBA029268669001,da found.flipping to correct cti.\r\n\r\nda de...,da reputation - behavioral - dumping package(s...,additional details: property manager calling i...,locker,locker,True,True
283,313931908,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA029728102301,contact id: a2t3nb9xdnr0lu\r\n\r\nattached pod...,da reputation - behavioral - dumping package(s...,additional details: pm reported packages are l...,locker,locker,True,True
501,313874884,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA030065579101,resolving ticket as not a tier infraction as p...,da reputation - behavioral - dumping package(s...,additional details: property manager said that...,locker,locker,True,False
552,313412547,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA026808760201,as per contact id: packages that were left by ...,da reputation - behavioral - dumping package(s...,additional details:the driver dumped all the p...,locker,locker,True,True
686,314210256,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA031342815701,"as per comp/psui tba031635485601,tba0313215625...",da reputation - behavioral - dumping package(s...,additional details: property manager explained...,locker,locker,True,False
758,314199368,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA023872139701,contact id: a2fp28jkcch6gj\r\n\r\nduplicate ti...,da reputation - behavioral - dumping package(s...,additional details: packages dumped outside l...,locker,locker,True,True
766,314166417,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA030992148801,as per update there are notes to deliver the p...,da reputation - behavioral - dumping package(s...,additional details: da left customers package ...,locker,locker,True,False
813,314167204,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA030200365001,contact id:a8qskeuit7wyk\r\n\r\nduplicate tick...,da reputation - behavioral - dumping package(s...,additional details:driver left the package on ...,locker,locker,True,False
818,314188386,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA032274783701,contact id: apdnoe27w10f\r\n\r\nduplicate tick...,da reputation - behavioral - dumping package(s...,additional details: da dumped customer package...,locker,locker,True,False
823,314188670,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - dumping packages outside locker,TBA031095649401,contact id: a22cdho6goz0b1\r\n\r\nattached pod...,da reputation - behavioral - dumping package(s...,additional details: driver left the package ou...,locker,locker,True,True


# in_or_on_a_USPS_mail_box

In [7]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('delivering in or on a usps mail box')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()


df['Second_Status']=df['Additional_Details'].str.contains('left the package in the usps mailroom|in the united state postal service|under his mailbox|inside usps mailbox|top of the usps mailbox|left the package at the mailbox|in the united state postal service|delivered it on top of mailbox|packages were found in the usps mail box|delivered to usp|found it on my mailbox|inside of usps mailbox|in usps mailbox|top of the community mail boxes|to usps mail box|in usps mail box|to a us mailbox|delivered on mailbox|left in a mail slot|in my mail box|in the usps mailbox|in our mailbox|to the usps mailbox|atop the communal mailboxes|inside mailbox|in usps mail box|in the usps mailbox|at the usps mailbox|in usps mailbox|in the customer mailbox|in the USPS mailbox|on the mailbox|in mailbox|inside a usps mail box|inside the mailbox|in the usps mail box|top of the mail box|in the mailbox|to postal mailbox|to the mailbox|to usps mailbox|inside the usps mailbox|it in my mailbox|top of the street mail box|on the usps mail box|on usps mail box|inside the mail box|on top of the usps mail box|left in a mail box|delivered to the mailbox|to customer''s mailbox|to mail box|to mailbox|in the united state postal service (usps) mail box|in united state postal service (usps) mail box|to the united state postal service (usps) mail box|to the united state postal service mail box|to the usps mailbox|top of the mailbox|in mailbox|in the post office|in the mail box|into my mail box|to usps reserved mailbox|to the usps mail box')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Delivering in or on a USPS mail box.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df

C:\Users\majidmm\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
21,313935429,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA030510084201,contact id: addrebtjexmbu\r\n\r\nduplicate tic...,da reputation - package mishandling - deliveri...,additional details: customer complained driver...,mail box,mail box,True,True
96,313909681,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA030058701101,"as per the contact id:a36nq92bygvcqz,driver de...",da reputation - package mishandling - deliveri...,additional details:driver delivered all of the...,mail box,mail box,True,True
135,313903729,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA030576126701,contact id: a103epxgmrrenj\r\n\r\nattached pod...,da reputation - package mishandling - deliveri...,additional details: customer says item was han...,mail box,mail box,True,True
142,313911173,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA030218398101,contact id: a63suojte34h8\r\n\r\nattached pod ...,da reputation - package mishandling - deliveri...,additional details: it was reported that this ...,mail box,mail box,True,True
275,313912363,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA029294209001,tier 2:da reputation - package mishandling - d...,da reputation - package mishandling - deliveri...,additional details:package delivered to mailbo...,mail box,mail box,True,True
513,313934373,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA026929619801,contact id: a1786ce4i4toic\r\n\r\nduplicate ti...,tier 2 da reputation - package mishandling - d...,additional details: customer calling because t...,mail box,mail box,True,True
565,314213274,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA032340247801,contact id: ahzg00dvh0pmb\r\n\r\nduplicate tic...,tier 2 da reputation - package mishandling - d...,additional details: package was place inside t...,mail box,mail box,True,False
635,314185969,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA033110118501,contact id:ag6znhgg0ddnc\r\n\r\n\r\nduplicate ...,da reputation - package mishandling - deliveri...,additional details:package was delivered to a ...,mail box,mail box,True,False
757,314185834,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA006964914801,contact id:aqprmbgfd58w8\r\n\r\nattached pod (...,da reputation - package mishandling - deliveri...,NaN,mail box,mail box,True,NaN
769,314098170,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - delivering in or on...,TBA027495479501,contact id: a35y1pn4mdvnqt\r\n\r\nattached pod...,tier 2 da reputation - package mishandling - d...,additional details:customer called saying thei...,mail box,mail box,True,False


# Left in Lawn

In [8]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('left in lawn')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

df['Second_Status']=df['Additional_Details'].str.contains('should not place the packages in the rain|under the tree|under the mailbox|outside his gate|front of mailbox|in the alley|by the mail box|next to the mailbox|left in the plants|walk way|near mailbox|under the mailbox|left the package in the garden|in the tree|in leaves|on planter|lawn|pond|sidewalk|grass|walkway|yard|side walk|water|bush|flower bed|left package at gate|left it in the front gate|river bed|on the ground|snow')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left_in_lawn.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)

df


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
71,313878184,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA029989815601,contact id: a2jw2tin0wwo0b\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: customer complains that th...,lawn,lawn,True,True
186,313887522,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA029900470601,"reviewed the contact id: a1k0lo283vn36l, custo...",da reputation - package mishandling - package(...,additional details: customer complains that th...,lawn,lawn,True,True
359,313937331,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA030146390301,contact id: a12u98afvyagr2\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: driver left the package at...,lawn,lawn,True,True
676,314203533,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA031670160201,contact id:https://tt.amazon.com/0314203533\r\...,da reputation - package mishandling - package(...,additional details:driver left the package on ...,lawn,lawn,True,True
716,314217852,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA032715277501,contact id: a1ix4zzl6k9hut\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: customer says my items wer...,lawn,lawn,True,True
721,314208874,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA032263445801,contact id: a1a3gr1u6bbi0r\r\n \r\nattached po...,da reputation - package mishandling - package(...,additional details: customer says \the driver ...,lawn,lawn,True,True
970,314206984,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA032423426601,contact id: a2z3zusrehmx0i\r\n \r\nattached po...,da reputation - package mishandling - package(...,additional details: this package was left on t...,lawn,lawn,True,True
986,314196435,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA024774320401,contact id: a2nhdbtnvgk3ry\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: customer says \the driver ...,lawn,lawn,True,True
999,314085353,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA030104357001,contact id: aohmffkm5dqo6\r\n\r\nattached pod ...,da reputation - package mishandling - package(...,additional details: he package was delivered j...,lawn,lawn,True,True
1040,314212330,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in lawn,TBA030134097301,contact id: a3qmeaqstazayh\r\n\r\nduplicate ti...,tier 2 da reputation - package mishandling - p...,additional details: driver left the package on...,lawn,lawn,True,True


# Left in road-curb

In [9]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('package mishandling - left in road/curb')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list=('pkg out in the street','left the packages near the road','outside of the gate','left them near road','left the package on sidewalk','left the package at the gate','left in the alley','near the road','left the package at gate','by the road','next to a telephone pole','by the mailbox','side the road','on the side walk','public sidewalk','by sidewalk','left out in the street','side of the street','next to the street','side of the road','beside the road','walkway','Package outside the Gate','at the main street','at the street','by the gate near the street','by the rain','by the street','curb','curb of the street','exposed to rain','from the street',
    'in the rain','left in the street','left on the ground in front of my house near the street','left the package almost in the street','left the package at street','left the package down the street','left the package in the street','middle of a busy street','middle of the street','of the rain',
    'on the curb in street','on the ground near the street','on the main street','on the road','on the sidewalk','on the street','outside a busy street','outside the gate','package to the street','raining')


df['attribute'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list)), 
                        expand=False).str.lower()

df['Second_Status']=df['Additional_Details'].str.contains('pkg out in the street|left the packages near the road|outside of the gate|left them near road|left the package on sidewalk|left the package at the gate|left in the alley|near the road|left the package at gate|by the road|next to a telephone pole|by the mailbox|side the road|on the side walk|public sidewalk|by sidewalk|left out in the street|side of the street|next to the street|side of the road|beside the road|walkway|curb|in the rain|on the road|exposed to rain|of the rain|raining|by the rain|outside a busy street|on the street|middle of the street|at the street|left the package down the street  |left the package almost in the street|Package outside the Gate| by the street|on the main street | on the curb in street  |package to the street|on the sidewalk|at the main street  |outside the gate|from the street|left on the ground in front of my house near the street|by the street|curb of the street|left the package in the street|on the ground near the street|by the gate near the street|left the package at street|middle of a busy street|left in the street')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left_in_road_curb.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute,Second_Status
202,313932582,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA030420063601,contact id: a149jussuzj0c0\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: customer complained driver...,left in road,NaN,NS,NaN,False
271,313934027,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA029910494801,contact id: a35nr60swu8h92\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: driver left my package at ...,left in road,NaN,NS,on the street,True
421,313899658,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA030051142901,contact id: a1awcqj2axfnzt\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: driver delivered the packa...,left in road,NaN,NS,NaN,False
598,314200756,Delivery Station: DTO5 Contact ID: A177L6...,t2 - package mishandling - left in road/curb,NaN,customer stated that the driver left the packa...,da reputation - package mishandling - package(...,additional details:customer stated that the dr...,left in road,NaN,NS,outside the gate,True
638,314090644,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA031442248101,contact id: a2uwxbb83iabej\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: cx complains driver left p...,left in road,NaN,NS,NaN,False
675,314215813,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA031543975801,contact id:al8ej4ta7odjo\r\n\r\nduplicate tick...,da reputation - package mishandling - package(...,additional details:the amazon driver just dump...,left in road,NaN,NS,curb,True
753,314193864,Copy and paste the Amazon Flex delivery partne...,t2 - package mishandling - left in road/curb,TBA033026150401,"driver found to be dsp da, flipping back to co...",tier 2 da reputation - package mishandling - p...,additional details:the customer says the drive...,left in road,NaN,NS,outside the gate,True
759,314176305,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA030798887201,contact id: a2ek548f8728mq\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details:the driver dropped outside ...,left in road,NaN,NS,outside the gate,True
781,314218129,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA032600169201,contact id: a1h587kfas2sst\r\n\r\nduplicate t...,da reputation - package mishandling - package(...,additional details: customer states: he went t...,left in road,NaN,NS,NaN,False
874,314186995,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in road/curb,TBA031721975601,contact id:a2n2kfou2cca6j\r\n\r\nduplicate tic...,da reputation - package mishandling - package(...,additional details: customer states that drive...,left in road,NaN,NS,on the sidewalk,True


# Left on near garbage or recycling

In [10]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('left on/near garbage or recycling')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list_1=('left','dump','put','deliver')

word_list_2=('dumpster','garbage','trash','recycling bin')


df['attribute_1'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list_1)),expand=False).str.lower()
df['attribute_2'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list_2)),expand=False).str.lower()

df['Second_Status']=(df['Additional_Details'].str.contains('left|dump|put|deliver'))&(df['Additional_Details'].str.contains('trash|dumpster|garbage|recycling bin'))

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_near garbage or recycling.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df




,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute_1,attribute_2,Second_Status
62,313912799,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA029273874201,contact id: a2lfi9qedizxq3\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: packages were left in recy...,on/near garbage,on/near garbage,True,left,recycling bin,True
82,313906317,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA029527251101,contact id: axd1pe17e6w4m\r\n \r\nattached pod...,da reputation - package mishandling - package(...,additional details: customer says \the driver ...,on/near garbage,on/near garbage,True,left,garbage,True
127,313834158,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA028670251901,note:- customer did not mention about scan mad...,da reputation - package mishandling - package(...,"additional details: customer stated that , the...",on/near garbage,on/near garbage,True,left,trash,True
350,313915386,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA029589706201,contact id:a1cijl3wiusscc\r\n\r\n\r\nduplicate...,da reputation - package mishandling - package(...,additional details: driver left the package ou...,on/near garbage,on/near garbage,True,left,trash,True
801,314166087,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,NaN,tba : tba031515919601\r\n\r\nas per call recor...,da reputation - package mishandling - package(...,additional details:the package got delivered n...,on/near garbage,on/near garbage,True,deliver,trash,True
1560,314046813,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA030168455501,contact id:am8xgfmg03rv4\r\n\r\nduplicate tick...,da reputation - package mishandling - package(...,additional details:the driver left the package...,on/near garbage,on/near garbage,True,left,garbage,True
2074,313659924,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA023665740801,contact id:a2ufzt3rt76jvp\r\n\r\nduplicate tic...,da reputation - package mishandling - package(...,additional details:customer complaint is that ...,on/near garbage,on/near garbage,True,left,trash,True
2148,313705467,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA029136313501,"reviewed the contact id: aex9ocf89qri1, custom...",da reputation - package mishandling - package(...,additional details: customer complains that t...,on/near garbage,on/near garbage,True,left,garbage,True
2468,313756867,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA027068478901,contact id: a308thjhvudodq\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: da left multiple packages ...,on/near garbage,on/near garbage,True,left,trash,True
2600,313688026,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left on/near garbag...,TBA029253618001,contact id: a2y3ty0q0cirto\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: item was left between tras...,on/near garbage,on/near garbage,True,left,trash,True


# Package Mishandling - Throwing

In [11]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('package mishandling - throwing/tossing')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list=('chunked','flip','kick','threw','throw','toss')

df['attribute'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list)),expand=False).str.lower()


df['Second_Status']=df['Additional_Details'].str.contains('chunked|kick|toss|throw|threw|flip')



df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Package Mishandling - Throwing.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df




,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute,Second_Status
5,313935185,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA031134532401,contact id: a180l2ypvdb3ot\r\n\r\nattached pod...,da reputation - package mishandling - throwing...,additional details: the customer states the th...,throwing,throwing,True,threw,True
9,313814108,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA029056999901,contact id:a1prv671virc60\r\n\r\nduplicate tic...,da reputation - package mishandling - throwing...,additional details: customer is stating that t...,throwing,throwing,True,threw,True
15,313915685,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA030550247501,contact id: a2tibdtvejx3dh\r\n\r\nduplicate ti...,da reputation - package mishandling - throwing...,additional details: customer said driver just ...,throwing,throwing,True,threw,True
52,313811445,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA029457209501,contact id:a3s2v6i6dxf7bv\r\n\r\nduplicate tic...,tier 2 da reputation - package mishandling - t...,additional details:the customer says the drive...,throwing,throwing,True,threw,True
115,313927089,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA030659975501,contact id: a2r3tc2ix85fsi\r\n\r\nduplicate ti...,da reputation - package mishandling - throwing...,additional details: da threw the package witho...,throwing,throwing,True,threw,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3368,314353394,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA033654497601,no data available,ailabl,NaN,throwing,NaN,NS,NaN,NaN
3429,314348637,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA033380782601,no data available,ailabl,NaN,throwing,NaN,NS,NaN,NaN
3453,314348005,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA033793716401,no data available,ailabl,NaN,throwing,NaN,NS,NaN,NaN
3455,314358909,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - throwing/tossing,TBA033354281501,no data available,ailabl,NaN,throwing,NaN,NS,NaN,NaN


# Package Mishandling Left in driveway

In [12]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('left in driveway')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()


df['Second_Status']=(df['Additional_Details'].str.contains('leave|kept|put|drop|leaving|stash|left|deliver|dumped|leave|abandon|sent'))&(df['Additional_Details'].str.contains('package|item|box| '))&(df['Additional_Details'].str.contains('against|under|unsafe|outside|at|middle|between|in|by|behind|of|on'))&(df['Additional_Details'].str.contains('car|dirt|windshield|roof of the truck|drive way|car tyre|two vehicles|garage|driveway|top of customer''s tire|under car|under car''s tyre|under customers tyre|behind the car|on car tyre|under customer''s tire|under the car|under car''s tire|parking lot|on customer''s husband''s car|under the car''s tyre|on the car trunk|left in a vulnerable state|outside the gate|on car|on top of a car|under customers truck|hood of the truck|truck tire|top of a car'))


df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left in driveway.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
41,313943007,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA030579139801,tier 2:da reputation - package mishandling - p...,da reputation - package mishandling - package(...,additional details:amazon packages keep being ...,left in driveway,left in driveway,True,True
43,313918368,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA030351262201,contact id: a1355sar9dwpbv\r\n\r\nattached po...,da reputation - package mishandling - package(...,additional details: customer contacted support...,left in driveway,left in driveway,True,True
85,313932770,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA030420023301,contact id: a34ef3hmxqkc28\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: customer: \it was left by ...,left in driveway,left in driveway,True,True
352,313892749,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA031033019501,contact id:ap9y2m27i1aya\r\n\r\nduplicate tick...,da reputation - package mishandling - package(...,additional details:the driver came flying up i...,left in driveway,left in driveway,True,False
405,313926919,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA019815871701,contact id: a21ijzprcab9lv\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: customer states that the d...,left in driveway,left in driveway,True,True
448,313933652,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA029238555301,contact id:a1jdnjuln3hc9c\r\n\r\nduplicate tic...,da reputation - package mishandling - package(...,additional details:the driver left the package...,left in driveway,left in driveway,True,False
508,313887328,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA029975458101,contact id: a38oqzlx8d5spf\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: the customer stating that ...,left in driveway,left in driveway,True,True
514,313934006,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA030504965301,reviewed contact an11g8xfj861s customer report...,da reputation - package mishandling - package(...,additional details: customer complained driver...,left in driveway,left in driveway,True,True
521,313931715,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA029751499201,contact id: a2onwd9ubi7q17\r\n\r\nattached pod...,da reputation - package mishandling - package(...,additional details: driver left the package in...,left in driveway,left in driveway,True,True
536,313931712,"Delivery Station (Station Code, City, State): ...",t2 - package mishandling - left in driveway,TBA030509170201,contact id: a21loisoegsd1r\r\n\r\nduplicate ti...,da reputation - package mishandling - package(...,additional details: my package was delivered t...,left in driveway,left in driveway,True,True


# property_damage

In [13]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('property damage')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list=('damage','destroy','rut','hit','tear','tire','pulled off','stain','tore','leak','knock','dent','damaging','ditch','tracks','ruin','mess','mud','break','broke','stuck','drove all over','driver drove','driven into','scratch','grass','hole','torn','ran over','drove over')


df['attribute'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list)), 
                        expand=False).str.lower()

df['Second_Status']=df['Additional_Details'].str.contains('banged|ripped|crush|damage|destroy|rut|hit|tear|tire|pulled off|stain|tore|leak|knock|dent|damaging|ditch|tracks|ruin|mess|mud|break|broke|stuck|drove all over|driver drove|driven into|scratch|grass|hole|torn|ran over|drove over')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_property_damage.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)

df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute,Second_Status
3,312949075,Your login (without @amazon): kimmears Co...,t2 - unsafe driving behavior - property damage,TBA022025095601,no data available,ailabl,NaN,property damage,NaN,NS,NaN,NaN
7,313907202,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - property damage,TBA021206316001,contact id: a1uc358n1irzn8\r\n\r\nattached pod...,da reputation - behavioral - property damage\r\n,additional details: customer contacted suppor...,property damage,property damage,True,damage,True
39,313910863,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - property damage,TBA029053040801,contact id: a1vfzxigqej7fq\r\n\r\nattached pod...,da reputation - behavioral - property damage\r\n,additional details: customer said that the dri...,property damage,property damage,True,damage,True
130,313809337,Who reported the incident?: Customer / Witness...,t2 - unsafe driving behavior - property damage,TBA029815306401,contact id: a2vtwpxvfs17ng\r\n\r\nduplicate ti...,da reputation - unsafe driving behavior - prop...,additional details: customer states: driver hi...,property damage,property damage,True,hit,True
156,313936780,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior - property damage,TBA030254123001,contact id: audjoc369g8xg\r\n\r\nattached pod ...,da reputation - unsafe driving behavior - prop...,additional details: customer says \the driver ...,property damage,property damage,True,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3525,314314327,Who reported the incident?: Customer / Witness...,t2 - unsafe driving behavior - property damage,TBA031864863001,no data available,ailabl,NaN,property damage,NaN,NS,NaN,NaN
3532,313804799,Who reported the incident?: Driver Statio...,t2 - unsafe driving behavior - property damage,TBA029268301301,contact id:a25q8qhs4o0ue8\r\n\r\nduplicate tic...,da reputation - unsafe driving behavior - prop...,additional details:driver delivered a package ...,property damage,property damage,True,damaging,True
3536,311490262,Your login (without @amazon): ausbandy Cop...,t2 - unsafe driving behavior - property damage,TBA008093195301,* this ticket will automatically be set to ass...,da reputation - unsafe driving behavior - prop...,additional details: customer states that they ...,property damage,property damage,True,hit,True
3540,E105305483,NaN,t2 - unsafe driving behavior - property damage,NaN,no data available,ailabl,NaN,property damage,NaN,NS,NaN,NaN


# Scanned business closed

In [14]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('scanned business closed')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()



df['Second_Status']=df['Additional_Details'].str.contains('when the school was open|address is the post office and it was not closed but driver incorrectly scanned as business closed|business closed for an apartment|business was not closed when driver attempted the delivery|driver scanned as business closed when the business open|business and the front door was open|business is very open|business was closed but it was open|business closed while it''s opened at the time of delivery|business closed when it was open|waiting for this order|business is still open|business is open|business was open|business is not closed|there are people|building is open|business hours have been updated')

# df['Scan_Status']=df['Reason'].str.contains('BUSINESS_CLOSED')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Scanned business closed.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)
df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status


# Scanned unable to access

In [15]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('scanned unable to access')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()



df['Second_Status']=df['Additional_Details'].str.contains('does not require any code|nothing to block access to the home|residential address|residential properties|falsely marked packages uta|customer provide driver with code driver scan customer package as unable to access|customer mentioned there is no dog, fence, gate nor construction going|no access code in the address|no access needed|there is no access code|there are no access codes|address is residential|never attempted to deliver|access is not needed|no access code required|no access code needed|effort was not made|unlocked door|no access barrier|notes has access code|provided instruction|not need any access code|door is always open|no access required|no access problem|no access code that is needed|no access code is required|residential home|no access code to front door')



df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Scanned unable to access.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)
df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status


# unattended in mailroom

In [16]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('behavioral - left packages unattended in mailroom/leasing')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list_1=('abandon','dump','outside','unattended','left')

word_list_2=('leasing','lobby','mail box','mail room','mail-room','mailbox','mailroom','parcel room')

df['attribute_1'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list_1)), 
                        expand=False).str.lower()

df['attribute_2'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list_2)), 
                        expand=False).str.lower()

df['Second_Status']=(df['Additional_Details'].str.contains('deliver|left|unattended|dump|outside|abandon|leave|top'))&(df['Additional_Details'].str.contains('office|leasing|mailroom|mail room|mailbox|mail box|parcel room|mail-room|lobby'))

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_unattended_in_mailroom.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df




,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute_1,attribute_2,Second_Status
37,E105794357,NaN,t2 - behavioral - left packages unattended in ...,NaN,tba026189110501 shows instructions in psui to ...,tier 2 da reputation - da reputation \u2013 be...,additional details:da leaving packages in comm...,mailroom/leasing,mailroom/leasing,True,NaN,NaN,False
369,313883089,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA028727487701,contact id:a1a0u3wk2qbb5j\r\n\r\nduplicate tic...,tier 2 da reputation - da reputation \u2013 be...,additional details:the property manager says t...,mailroom/leasing,NaN,NS,left,leasing,True
814,314198124,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA032221302001,contact id: aq7ch5zs5bt9g\r\n\r\nattached pod ...,da reputation - da reputation \u2013 behaviora...,additional details: customer said his package ...,mailroom/leasing,mailroom/leasing,True,dump,mailroom,True
922,314218842,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA028935985401,no data available,ailabl,NaN,mailroom/leasing,NaN,NS,NaN,NaN,False
1328,314054854,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA029308644001,contact id:az97mjnt6o7oj\r\n\r\nattached pod (...,da reputation - da reputation \u2013 behaviora...,additional details:driver leaving packages in ...,mailroom/leasing,mailroom/leasing,True,NaN,NaN,False
1421,314038858,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA030768998701,did you consult the available (tm/rs) on the f...,da reputation - da reputation \u2013 behaviora...,additional details: customer provided prior in...,mailroom/leasing,mailroom/leasing,True,left,mail room,True
1674,314062108,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA029292638201,contact id:a2lppjl25ywf7l\r\n\r\nduplicate tic...,tier 2 da reputation - da reputation \u2013 be...,additional details:the property manager says t...,mailroom/leasing,mailroom/leasing,True,dump,leasing,True
1720,314054892,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA029860650101,contact id:a16gl4yg0s6os2\r\n\r\nduplicate tic...,tier 2 da reputation - da reputation \u2013 be...,additional details:the property manager stated...,mailroom/leasing,mailroom/leasing,True,left,mailroom,True
1736,314062247,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA222215126000,contact id:a2fcic3g7qusun\r\n\r\nduplicate tic...,tier 2 da reputation - da reputation \u2013 be...,additional details: the property manager state...,mailroom/leasing,mailroom/leasing,True,left,NaN,False
1983,313635868,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - left packages unattended in ...,TBA025472766401,contact id:a9gz95kanq15t\r\n\r\nduplicate tick...,tier 2 da reputation - da reputation \u2013 be...,additional details: this poc has had reoccurri...,mailroom/leasing,mailroom/leasing,True,dump,mail room,True


# Unauthorized entry of entryway garage

In [17]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('behavioral - unauthorized entry of entryway/garage/foyer')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list=('entered home|past the locked entry door','driver entered the house','driver enter her back yard','driver opened my basement door','staring through the windows','driver entered the customer''s house','driver hopped over the customer''s fence','driver entered into the side gate','entered customer back yard','driver opened the front door','entered customer''s garage','driver was found in in his open garage','enter his house','enter the back gate','enter the customer''s house','enter the customers house','entered customer''s garage','entered the gate','entered through a side door','entered through the backyard','entered to leave the package',
    'entering into the house','into the customers house','jumped over the fence','looking around','open her door','open the customer front door','open the customer''s back door','open the customer''s door','open the door','open the front door',
    'opened front door','opened the door of customer','opening the door','over the gate','pass the no passing zone','trespass','walk around','wander','without customer''s permission','without her permission')


df['attribute'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list)),expand=False).str.lower()


df['Second_Status']=df['Additional_Details'].str.contains('permission|opened his back door|opened customer''s garage|trying to enter|spying|pull the customer door|jump the fence|walked into|inside the house|left fence open|entered his garage|came in her house|all the way around my house to the back|straight into the door|came inside the house|attempted to open|came into my garage|entered home|driver entered her home|past the locked entry door|driver entered the house|driver enter her back yard|driver opened my basement door|staring through the windows|driver entered the customer''s house|driver hopped over the customer''s fence|driver entered into the side gate|driver opened the front door|entered customer back yard|entered customer''s garage|enter his house|open the door|jumped over the fence|enter the back gate|trespass|walk around|without customer''s permission|entering into the house|over the gate|opening the door|driver was found in in his open garage|entered the gate|entered through the backyard|open her door|open the front door|open the customer''s door|entered through a side door|opened front door|into the customers house|open the customer''s back door|without her permission|enter the customer''s house|looking around|open the customer front door|pass the no passing zone|opened the door of customer|entered to leave the package|enter the customers house|wander')



df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unauthorized entry of entryway garage foyer.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute,Second_Status
578,314183229,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA031474094001,contact id: ax9eiuaxgpzlt\r\n\r\nduplicate tic...,da reputation - behavioral - actual or attempt...,additional details: customer states that the d...,unauthorized entry,unauthorized entry,True,NaN,True
773,314213225,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA032159632101,note:cx states that \xa8the driver went inside...,da reputation - behavioral - actual or attempt...,additional details: the driver opened the cust...,unauthorized entry,unauthorized entry,True,NaN,False
777,314215628,Your login (without @amazon): jennifik Co...,t2 - behavioral - unauthorized entry of entryw...,TBA032469513201,i checked street view of the house in bing map...,da reputation - behavioral - actual or attempt...,additional details: customer stated that driv...,unauthorized entry,unauthorized entry,True,trespass,True
891,314086032,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA032289081401,contact id:a3q302cu0vnwqt\r\n\r\n \r\n\r\natta...,da reputation - behavioral - actual or attempt...,additional details:driver was looking through ...,unauthorized entry,unauthorized entry,True,NaN,False
1122,314168339,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA032503970801,contact id: a36ir0z1lizp5o\r\n\r\nattached pod...,da reputation - behavioral - actual or attempt...,additional details: customer says that just re...,unauthorized entry,unauthorized entry,True,NaN,False
1716,313964549,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA028981256901,**customer stated that driver delivered the pa...,da reputation - behavioral - actual or attempt...,additional details: driver came back in the cu...,unauthorized entry,unauthorized entry,True,NaN,False
1799,313685921,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA027864271401,the customer says the driver came inside the f...,da reputation - behavioral - actual or attempt...,additional details:the customer says the drive...,unauthorized entry,unauthorized entry,True,NaN,False
1867,313674787,Your login (without @amazon): yumikfra Co...,t2 - behavioral - unauthorized entry of entryw...,TBA027236121001,customer chatted in to report driver had place...,tier 2 da reputation - behavioral - actual or ...,additional details:customer stated that the dr...,unauthorized entry,unauthorized entry,True,NaN,True
2335,313689741,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA028382199601,contact id:a399wwm6vawu1d\r\n\r\nduplicate tic...,tier 2 da reputation - package mishandling - p...,additional details:the customer says the drive...,unauthorized entry,on/near garbage,NS,NaN,False
2441,313796812,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unauthorized entry of entryw...,TBA028924811101,customer complained that the driver opened the...,da reputation - behavioral - actual or attempt...,additional details: customer complained that t...,unauthorized entry,unauthorized entry,True,NaN,True


# Unprofessional behavior towards a customer

In [18]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('unprofessional behavior towards a customer')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()


df['Second_Status']=df['Additional_Details'].str.contains('preaching religion|he did not feel like delivering the packages door to door|arguing|told the customer to go to him|nasty|argument|driver shout|obscene language|banged the customer door|driver flipped customer off|receive the package from his car|beg|inappropriate|accuse|harass|aggressive|shook the door|slam|pee|driver took a picture of his father|driver not getting out of the truck|don''t order|deal with it|asked for food and money|uncomfortable|argue|bad tone|verbal altercation|rant|not polite|abuse|bad word|offensive|offensive verbal exchange|cussing|improper words|driver behaved improperly|disrespect|bad language|profane language|yell|attitude|unprofessional|remark|rude|curse|middle finger|abusive|scream|cursing|profanity|banging|abusing|threatened|intimidating|disrespectful|foul language|honking the horn constantly|i will not deliver the package to you next time|mis-gender')


df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unprofessional behavior towards a customer.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
1,312963401,Copy and paste the Amazon Flex delivery partne...,t2 - behavioral - unprofessional behavior towa...,NaN,driver is a da. flipping to correct cti. \r\n\...,"da reputation - behavioral - insulting, unprof...",NaN,behavior towards a customer,NaN,NS,NaN
19,313885445,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unprofessional behavior towa...,TBA028052141901,**issue given that driver was rude while deliv...,"da reputation - behavioral - insulting, unprof...",additional details:customer mentioned driver ...,behavior towards a customer,NaN,NS,True
31,313931642,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unprofessional behavior towa...,TBA029042132501,contact id: ac0fmwke09i94\r\n\r\nduplicate tic...,"tier 2 da reputation - behavioral - insulting,...","additional details: customer states, i receiv...",behavior towards a customer,NaN,NS,False
227,313926750,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unprofessional behavior towa...,TBA221736320000,contact id: a33084ey52sfvq\r\n\r\nattached pod...,"da reputation - behavioral - insulting, unprof...",additional details: customer states the driver...,behavior towards a customer,NaN,NS,True
239,313808089,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unprofessional behavior towa...,TBA030334144801,contact id: alp98vmfsuzrh\r\n\r\nduplicate tic...,"da reputation - behavioral - insulting, unprof...",additional details: customer complained that d...,behavior towards a customer,NaN,NS,True
...,...,...,...,...,...,...,...,...,...,...,...
3297,313160885,Your login (without @amazon): ausbandy Cop...,t2 - behavioral - unprofessional behavior towa...,NaN,please find video and screenshot of original p...,"da reputation - behavioral - insulting, unprof...","additional details:the delivery driver saying,...",behavior towards a customer,NaN,NS,False
3399,314357730,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - unprofessional behavior towa...,NaN,no data available,ailabl,NaN,behavior towards a customer,NaN,NS,NaN
3549,312895264,Your login (without @amazon): aquillag Co...,t2 - behavioral - unprofessional behavior towa...,TBA021560957001,\u0001f4e6\u0001f4e6\u0001f4e6\u0001f4e6sds~ec...,"tier 2 da reputation - behavioral - insulting,...","additional details:the wipes were delivered, h...",behavior towards a customer,NaN,NS,False
3550,312890442,Your login (without @amazon): jenarg@amazon.co...,t2 - behavioral - unprofessional behavior towa...,TBA022353286401,- sds ecr has reached out to the customer adv...,"da reputation - behavioral - insulting, unprof...",additional details: your driver just threw 2 b...,behavior towards a customer,NaN,NS,True


# Unsafe driving behavior

In [19]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')
df=df[df['root_cause'].str.contains('unsafe driving behavior')==True]
df=df[df['root_cause'].str.contains('unsafe driving behavior - property damage')==False]
df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

word_list=('wrong direction','drove on her lawn','drove on the yard','360 degrees turn','grass','drove on to the customer''s yard','mud','block','drove on their backyard','over 35 MPH','fly','smash','shatter','drove over','mess','cutting off traffic','drove over the customers lawn','spin','tire track','through her lawn','wrong side','drove by his front lawn','pace that was unsafe','360 in the parking','pulling in customers lawn','driving through the grass','drove on the customer lawn','rut','ran into','displace','damage','reckless','wrong way','destroy','60miles/hour','drove off our driveway sinking into our yard','fast','hit','knock','mark','speed','rush','drover over','drove off our driveway')


df['attribute'] = df.Additional_Details.str.extract('({})'.format('|'.join(word_list)),expand=False).str.lower()

df['Second_Status']=df['Additional_Details'].str.contains('slow down|wrong direction|drove on her lawn|drove on the yard|360 degrees turn|grass|drove on to the customer''s yard|mud|block|drove on their backyard|over 35 MPH|fly|smash|shatter|drove over|mess|cutting off traffic|drove over the customers lawn|spin|tire track|through her lawn|wrong side|drove by his front lawn|pace that was unsafe|360 in the parking|pulling in customers lawn|driving through the grass|drove on the customer lawn|rut|ran into|displace|damage|reckless|wrong way|destroy|60miles/hour|drove off our driveway sinking into our yard|speed|mark|hit|fast|knock|rush|drove off our driveway')

df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unsafe driving behavior.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)


df


,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,attribute,Second_Status
88,313936364,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA030006944201,contact id: a3b56lgn7jrm2i\r\n \r\nduplicate t...,da reputation \u2013 unsafe driving behavior\r\n,additional details:customer complains : \my de...,NaN,NaN,NS,hit,True
93,313916878,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA029733775401,reviewed contact adhb2xe2bi3se customer states...,da reputation \u2013 unsafe driving behavior\r\n,additional details: customer complained driver...,NaN,NaN,NS,reckless,True
481,313938926,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA031035146401,customer complained that the driver was speedi...,da reputation \u2013 unsafe driving behavior\r\n,additional details: customer complained that ...,NaN,NaN,NS,speed,True
519,313818902,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA029588619801,contact id: a3n8nfhgvul3gy\r\n\r\nattached pod...,da reputation \u2013 unsafe driving behavior\r\n,"additional details: customer stated that, the ...",NaN,NaN,NS,speed,True
586,314206740,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA009356978901,contact id:a3ilgkwqoaj2tg\r\n\r\nduplicate tic...,tier 2 da reputation \u2013 unsafe driving beh...,additional details:the customer says the drivi...,NaN,NaN,NS,speed,True
596,314212122,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA032582641601,contact id:a2fchu1mm6pwnn\r\n\r\nduplicate tic...,tier 2 da reputation \u2013 unsafe driving beh...,additional details:the customer says the drive...,NaN,NaN,NS,NaN,False
608,314206963,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA032097844801,contact id: a3cldlfjsvzft7\r\n\r\nduplicate ti...,da reputation \u2013 unsafe driving behavior\r\n,additional details: driver was speeding after ...,NaN,NaN,NS,speed,True
613,314178357,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA030398511501,contact id: a2pqgnoo9jmasd\r\n\r\nattached pod...,da reputation \u2013 unsafe driving behavior\r\n,additional details: customer said that the dri...,NaN,NaN,NS,NaN,False
672,314175692,Who reported the incident?: Customer / Witness...,t2 - unsafe driving behavior,TBA032593638401,contact id: a2cxanfp67mekl\r\n\r\nattached pod...,da reputation \u2013 unsafe driving behavior\r\n,additional details: customer has a round a bou...,NaN,NaN,NS,damage,True
856,314212143,"Delivery Station (Station Code, City, State): ...",t2 - unsafe driving behavior,TBA031545302001,contact id: a1pvqd6w0q4vnh\r\n\r\nduplicate ti...,da reputation \u2013 unsafe driving behavior\r\n,additional details: da was found speeding in c...,NaN,NaN,NS,speed,True


# Incorrectly scanned delivered to receiver

In [20]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('behavioral - incorrectly scanned delivered to receiver')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

# df['Status_1']=df['Additional_Details'].str.contains('no one was at home|customer didn''t sign for their package|item was not handed|package was never handed|customer was not in the address|no one was there to receive|package wasn''t handed|not deliver to receiver|not delivered to any receiver|delivered to receiver when it was not|left the packages outside|was never handed to her|is not at home|no one home|no one at the time of delivery|not handed directly to anyone|was left un attended|wasn''t handed to her|did not hand over the package to the customer|customer is not at home|was not left with the customer|left outside|customer was not home')


df['Second_Status']=(df['Additional_Details'].str.contains('package was never delivered|elevator|did not hand the package to the customer|no one was at home|customer didn''t sign for their package|item was not handed|package was never handed|customer was not in the address|no one was there to receive|package wasn''t handed|not deliver to receiver|not delivered to any receiver|delivered to receiver when it was not|left the packages outside|was never handed to her|is not at home|no one home|no one at the time of delivery|not handed directly to anyone|was left un attended|wasn''t handed to her|did not hand over the package to the customer|customer is not at home|was not left with the customer|left outside|customer was not home|did not hand over to the customer|has not happened|side walk|nobody was home|vending machine|no on was home|under the car|driver way|lawn|amazon hub|garbage|snow|outside|inaccurate delivery scan|alley|hallway|not true|parking lot|trash can|garage|planter bed|location|locker|yard|pile of snow|address|customers home|mail room|office|leasing|business area|mail box|bush|road|grass|porch|lock box|stair|unattended|building|gate|street|ground|driveway|desk|sidewalk|customer''s business|door|apartment|mailbox|floor|lobby|steps|front porch|bush|waste management|curb'))

df1=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\INPUT_FILES\comp_data.csv',encoding='ISO-8859-1')
df1.dropna(axis = 0, how = 'all', inplace = True) 

df2=df.merge(df1,left_on='tid',right_on='Tracking Id',how='left')


df2.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Incorrectly scanned delivered to receiver.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)
df2

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,...,TBA Deposit Code,Amazon Deposit Code,Route Sequence,sort_zone,packaging,sellerName,inductedCycle,assignedCycle,sideLineUntilTime,Estimated Arrival Date
0,313913853,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA030148222801,contact id: a1lofgcckkmj1c\r\n\r\nduplicate ti...,da reputation - behavioral - scanned delivered...,additional details: customer states that the d...,delivered to receiver,delivered to receiver,True,...,NaN,NaN,104.0,C-33.1B,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/11/2020 23:00
1,313911936,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA030284484801,reviewed contact aoiws6olkms03 customer states...,da reputation - behavioral - scanned delivered...,additional details: driver left the customer p...,delivered to receiver,delivered to receiver,True,...,NaN,NaN,26.0,C-20.2D,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/12/2020 20:00
2,313899064,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA030011210901,contact id: a3hqpcof8kyc7t\r\n \r\nduplicate ...,da reputation - behavioral - scanned delivered...,additional details: customer reached out comp...,delivered to receiver,delivered to receiver,True,...,NaN,NaN,14.0,D-13.1B,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/12/2020 22:00
3,313848460,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA028352887601,contact id:a1xf2jtfkrigeb\r\n\r\nduplicate tic...,da reputation - behavioral - scanned delivered...,additional details: customer is sttaing that t...,delivered to receiver,delivered to receiver,True,...,NaN,NaN,41.0,A-31.4D,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/11/2020 20:00
4,313812085,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA029701177801,contact id:a17tb6cujb2yat\r\n\r\nduplicate tic...,da reputation - behavioral - scanned delivered...,additional details:the customer contacted beca...,delivered to receiver,delivered to receiver,True,...,NaN,NaN,53.0,D-34.3B,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/11/2020 20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,314362099,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA034149715901,no data available,ailabl,NaN,delivered to receiver,NaN,NS,...,NaN,NaN,167.0,D-10.1B,No Packaging Need,NaN,CYCLE_2,CYCLE_1,NaN,5/15/2020 20:00
149,314398262,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,NaN,no data available,ailabl,NaN,delivered to receiver,NaN,NS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,314346275,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA032342631301,no data available,ailabl,NaN,delivered to receiver,NaN,NS,...,NaN,NaN,170.0,A-37.1E,No Packaging Need,NaN,CYCLE_1,CYCLE_1,NaN,5/14/2020 22:00
151,314395152,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrectly scanned delivere...,TBA031890246301,no data available,ailabl,NaN,delivered to receiver,NaN,NS,...,NaN,NaN,78.0,C-1.2E,No Packaging Need,NaN,CYCLE_2,CYCLE_1,NaN,5/13/2020 23:00


# Incorrect Scan

In [21]:
df=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_infractions_first_check.csv')

df=df[df['root_cause'].str.contains('behavioral - incorrect scan')==True]

df['Data']=df['Data'].str.lower() 
df['Infraction']=df['Infraction'].str.lower() 
df['Additional_Details']=df['Additional_Details'].str.lower()

df['Second_Status']=df['Additional_Details'].str.contains('and scanned the packages as|and left the package by the gate|instead it was delivered|and marked them as|which is incorrect|was not followed|but left at|scan incorrectly|incorrectly scan the package|but they left in|but left the package|incorrectly scanned|but left package at|but the packages were left|but left package at|instead of leaving it|instead marked scan as|but the packages were left|and marked package as|tried delivering the package to an incorrect address|however, package was delivered|did not deliver|scanned the package on the wrong location|when it was delivered|but left package|not at|did not leave the package|from far location|marked it incorrectly|it was actually delivered|away from customer|far away|outside the property|business closed before reaching|inaccurate|packages as business closed that were residential addresses|scanned it as|marked incorrectly|marked as|as scanned|marked it as|but the package was left|scanned as|which is not true|marked the scan as|marked the package as|scanned the package as')



df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Behavioral - Incorrect Scan.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)
df

,ShortId,details,root_cause,tid,Data,Infraction,Additional_Details,Infraction1,Infraction2,First_Status,Second_Status
164,313826011,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA029242428301,contact id: a1dooalvew96ow\r\n\r\nattached po...,da reputation \u2013 behavioral \u2013 incorre...,additional details: driver marked the scan as ...,incorrect scan,incorrect scan,True,True
228,313808284,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA027299894801,contact id:a2kq4nhs7aonlb\r\n\r\nduplicate tic...,da reputation \u2013 behavioral \u2013 incorre...,additional details: driver delivered the packa...,incorrect scan,incorrect scan,True,True
249,313930488,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA031214792401,contact id: a169vxj1res23q\r\n\r\nduplicate ti...,da reputation \u2013 behavioral \u2013 incorre...,additional details: customer states that the d...,incorrect scan,incorrect scan,True,True
278,313825566,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA028927838601,contact id:a2t5fwxmnm5iz3\r\n\r\nduplicate tic...,da reputation \u2013 behavioral \u2013 incorre...,additional details: driver delivered the packa...,incorrect scan,incorrect scan,True,True
304,313924171,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA030753247401,contact id: a2x8o1pjac1pna\r\n\r\nattached pod...,da reputation \u2013 behavioral \u2013 incorre...,additional details: driver scanned the package...,incorrect scan,incorrect scan,True,True
...,...,...,...,...,...,...,...,...,...,...,...
3152,314320110,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA033613690701,no data available,ailabl,NaN,incorrect scan,NaN,NS,NaN
3341,314356725,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA033800387601,no data available,ailabl,NaN,incorrect scan,NaN,NS,NaN
3353,314355074,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA033016927401,no data available,ailabl,NaN,incorrect scan,NaN,NS,NaN
3372,314406870,"Delivery Station (Station Code, City, State): ...",t2 - behavioral - incorrect scan,TBA033753795801,no data available,ailabl,NaN,incorrect scan,NaN,NS,NaN


# Collated Files except not a tier, duplicate, nfdi

In [22]:
df1=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Delivering in or on a USPS mail box.csv')
df2=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Dumping packages inside lobby.csv')
df3=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Dumping packages outside locker.csv')
df4=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left in driveway.csv')
df5=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left_in_lawn.csv')
df6=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_near garbage or recycling.csv')
df7=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Package Mishandling - Throwing.csv')
df8=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_property_damage.csv')
df9=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Scanned business closed.csv')
df10=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Scanned unable to access.csv')
df11=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_unattended_in_mailroom.csv')
df12=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unauthorized entry of entryway garage foyer.csv')
df13=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unprofessional behavior towards a customer.csv')
df14=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Unsafe driving behavior.csv')
df15=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Behavioral - Incorrect Scan.csv')
df16=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_Incorrectly scanned delivered to receiver.csv')
df17=pd.read_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\verified_left_in_road_curb.csv')




df = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17], ignore_index=True,sort=False)
df.to_csv(r'C:\Users\majidmm\Documents\MTI and RC\SAMPLE AUDIT\OUTPUT_FILES\Collated_end_infractions.csv',columns=('ShortId','tid','root_cause','Infraction','Additional_Details','First_Status','Second_Status'),index = False)
df

,ShortId,tid,root_cause,Infraction,Additional_Details,First_Status,Second_Status
0,313935429,TBA030510084201,t2 - package mishandling - delivering in or on...,da reputation - package mishandling - deliveri...,additional details: customer complained driver...,True,True
1,313909681,TBA030058701101,t2 - package mishandling - delivering in or on...,da reputation - package mishandling - deliveri...,additional details:driver delivered all of the...,True,True
2,313903729,TBA030576126701,t2 - package mishandling - delivering in or on...,da reputation - package mishandling - deliveri...,additional details: customer says item was han...,True,True
3,313911173,TBA030218398101,t2 - package mishandling - delivering in or on...,da reputation - package mishandling - deliveri...,additional details: it was reported that this ...,True,True
4,313912363,TBA029294209001,t2 - package mishandling - delivering in or on...,da reputation - package mishandling - deliveri...,additional details:package delivered to mailbo...,True,True
...,...,...,...,...,...,...,...
1065,313622652,TBA027825015101,t2 - package mishandling - left in road/curb,da reputation - package mishandling - package(...,additional details: customer complained driver...,NS,True
1066,312879150,TBA022494552401,t2 - package mishandling - left in road/curb,ailabl,NaN,NS,NaN
1067,314366730,TBA034084462301,t2 - package mishandling - left in road/curb,ailabl,NaN,NS,NaN
1068,314351262,TBA033705811301,t2 - package mishandling - left in road/curb,ailabl,NaN,NS,NaN
